# Information Aggregator

#### Author: Iain Muir, iam9ez@virginia.edu  
*Date: January 20th, 2021  
Project: Information Aggregator Application — powered by Datapane*

## Sources

* **[Robinhood](https://robin-stocks.readthedocs.io/en/latest/index.html)**
* **[Finnhub](https://finnhub.io/docs/api/)**
* **[ESPN](https://gist.github.com/akeaswaran/b48b02f1c94f873c6655e7129910fc3b)**
* **[The New York Times](https://developer.nytimes.com/apis)**
* **[CNN Money](https://money.cnn.com/data/markets)**
* **[FiveThirtyEight](https://data.fivethirtyeight.com/)**
* **[COVID Act Now](https://apidocs.covidactnow.org/api)**
* **[Lichess](https://lichess.org/api)**
* **[Spotify](https://developer.spotify.com/documentation/)**
* **[YouTube]()**

## Table of Contents <a class="anchor" id="toc"></a>

* **[0. Import Libraries and Secrets](#import)**  
* **[1. Build Report Components](#build)**  
    * [1.1 Header](#header)  
    * [1.2 Main Selector](#selector)  
        * *[1.2.1 Sports Results and Analytics](#espn)*  
        * *[1.2.2 Top World News](#nyt)*  
        * *[1.2.3 Stock Market](#finnhub)*  
        * *[1.2.4 Portfolio Performance](#robinhood)*  
        * *[1.2.5 Data Exploration](#data)*  
        * *[1.2.6 Compiled Selector](#compiled)*  
    * [1.3 Credits](#credits)  
* **[2. Datapane Report](#report)**  

## 0. Import Libraries and Secrets <a class="anchor" id="import"></a>

[Table of Contents](#toc)

Importing Standard Libraries...

In [1]:
from itertools import zip_longest
from bs4 import BeautifulSoup
from functools import partial
import datapane as dp
import pandas as pd
import numpy as np
import webbrowser
import warnings
import datetime
import requests
import time
import json

Importing Modules...

In [2]:
from robinhood import ticker_toggle, make_header, authenticate_, load_portfolio, get_scroll_objects, robinhood_news
from espn import format_news, news, parse_team, format_scores, scores, group_sport, standings
from finnhub import quote, candles, candlestick, name_search, big_number, profile, news
from nyt import top_stories, semantics, format_article, format_sections
from cnn_money import cnn_big_numbers, format_modules, group_modules
from fivethirtyeight import active_teams, make_button, plot_elo
from covid import county_choropleth, plot_state, altair_line
from spotify import get_spotify_embed

In [3]:
from errors import ErrorHandler, Logging, get_error_info 

In [4]:
from constants import ROOT, API_LOGOS, ESPN_SPORTS

Checking Datapane version...

In [5]:
version = !datapane --version

In [6]:
assert version[0].split()[2] == '0.13.2'

In [7]:
!datapane login --token=55010cebc170ecfbeddb82838c360776bf36f6be

Connected successfully to https://datapane.com as iainmuir


Open and Unpack Secrets File...

In [8]:
with open('secrets.json') as s:
    secrets = json.loads(s.read())

In [9]:
# Datapane
DATAPANE_KEY = secrets['datapane']
    
# Finnhub
FINNHUB_KEY = secrets['finnhub']

# Robinhood
ROBIN_USERNAME, ROBIN_PASSWORD = secrets['robinhood'].values()

# New York Times
NYT_KEY,  NYT_SECRET, NYT_ID = secrets['nyt'].values()

# Spotify
SPOTIFY_SECRET, SPOTIFY_ID = secrets['spotify'].values()

# COVID Act Now
COVID_KEY = secrets['covid']

In [10]:
AUTH_URL = 'https://accounts.spotify.com/api/token'
auth_response = requests.post(
    AUTH_URL, 
    {
        'grant_type': 'client_credentials',
        'client_id': SPOTIFY_ID,
        'client_secret': SPOTIFY_SECRET
    }
)
auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

In [11]:
try:
    robinhood = authenticate_(ROBIN_USERNAME, ROBIN_PASSWORD)
except KeyError:
    print(
        ErrorHandler("Failed Robinhood Authentication", *get_error_info())
    )
    robinhood = None

In [12]:
assert robinhood is not None

Preferences

In [13]:
warnings.simplefilter(action="ignore", category=pd.core.common.SettingWithCopyWarning)

## 1. Build Report Compenents <a class="anchor" id="build"></a>

[Table of Contents](#toc)

In [14]:
TODAY = datetime.date.today()

### 1.1 Header <a class="anchor" id="header"></a>

In [15]:
datapane, nyt, espn, spotify, _538, lichess = API_LOGOS.values()

In [16]:
header_logo = dp.HTML(
    """
    <html>
        <style type='text/css'>
            .images {
                display:flex;
                justify-content:center;
                align-items:center;
            }
            .images img {
                margin-left:5px;
                margin-right:5px;
            }
        </style>
        
        <center>
            <div class='images'>
                <img src='""" + datapane + """' width="75"/>
                <img src='""" + nyt + """' width="100"/>
                <img src='""" + espn + """' width="75"/>
                <img src='""" + spotify + """' width="75"/>
                <img src='""" + _538 + """' width="75"/>
                <img src='""" + lichess + """' width="75"/>
            </div>
        </center>
    </html>
    """
)

In [17]:
header_text = dp.HTML(
    """
    <html>
        <style type='text/css'>
            @keyframes rotate {
                0%   {color: #EEE;}
                25%  {color: #EC4899;}
                50%  {color: #8B5CF6;}
                100% {color: #EF4444;}
            }
            h1 {
                color:#eee;
                animation-name: rotate;
                animation-duration: 4s;
                animation-iteration-count: infinite;
            }
        </style>
        <center>
            <h1>Morning Scoop</h1>
            <i>""" + TODAY.strftime('%A, %B %d, %Y') + """<i>
        </center>
    </html>
    """
)

In [18]:
header_description = dp.Text(
    """
Welcome to the Morning Scoop! This project is aimed to aggregate and display an eclectic and wide-ranging mix of information, data, and visualizations. Information sources include The New York Times, ESPN, FiveThirtyEight, and many more. Enjoy!

This report is powered by Datapane.

    """.strip()
)

### 1.2 Main Selector <a class="anchor" id="selector"></a>

[Table of Contents](#toc)

#### 1.2.1 Sports Results and Analytics <a class="anchor" id="espn"></a>

Scores/Analytics/YouTube Highlights

In [19]:
try:
    espn_results = list(map(
        scores, ESPN_SPORTS.items()
    ))
    espn_results = list(map(
        lambda result: [dp.Text('No games played yesterday...')] if result == [] else result, espn_results
    ))
except:
    print(
        ErrorHandler("Failed to get ESPN Results", *get_error_info())
    )
    exit(0)

espn.py (Line: ~400) || CustomError: No Stats/Leaders for TCU @ ARIZ; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for OSU @ VILL; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for MIA @ AUB; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for MSU @ DUKE; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for TEX @ PUR; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for ND @ TTU; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for ISU @ WISC; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for HOU @ ILL; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for POR @ IND; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for MEM @ HOU; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for NO @ ATL; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for OKC @ ORL; OK.
espn.py (Line: ~400) || CustomError: No Stats/Leaders for PHX @ SAC; OK.
espn.py (Line: ~400) || CustomError: No Stats/Lea

Upcoming Schedule

In [20]:
try:   
    espn_upcoming = list(map(
        lambda sport: scores(sport, upcoming=True), ESPN_SPORTS.items()
    ))
    espn_upcoming = list(map(
        lambda game: [dp.Text('No upcoming games today...')] if game == [] else game, espn_upcoming
    ))
except:
    print(
        ErrorHandler("Failed to get ESPN Schedule", *get_error_info())
    )
    exit(0)

Standings

In [21]:
# records = list(map(
#     standings, ESPN_SPORTS.items()
# ))

##### Compiled ESPN Group

In [22]:
espn_groups = list(map(
    group_sport, ESPN_SPORTS.keys(), espn_results, espn_upcoming
))

#### 1.2.2 Top World News <a class="anchor" id="nyt"></a>

In [23]:
articles = top_stories(NYT_KEY)

In [24]:
articles = articles.loc[articles['item_type'] != 'Promo']
articles = articles.loc[articles['multimedia'].str.len() >= 1]
articles['multimedia'] = articles['multimedia'].str[0].str['url']
articles['subsection'] = articles['subsection'].replace('', 'miscellaneous')

In [25]:
grouped_articles = articles.groupby(
    'subsection',
    axis=0
)

In [26]:
ordered_groups = grouped_articles.size().sort_values(ascending=False)

In [27]:
article_sections = list(map(
    partial(format_sections, grouped_articles), ordered_groups.index
))

In [28]:
# TODO Article Links

#### 1.2.3 Stock Market <a class="anchor" id="finnhub"></a>

##### Ticker Scroll

In [29]:
t100 = robinhood.markets.get_top_100()
t100 = pd.DataFrame(t100)

In [30]:
scroll_objects = t100[['last_trade_price', 'previous_close', 'symbol']].apply(
    get_scroll_objects, axis=1
)

In [31]:
scroll = dp.HTML(
    """
    <html>
        <style type='text/css'>
            body {
                background: #FFFFFF;
            }
            
            .news-scroll a {
                font-size: 16px;
                text-decoration: none;
                color: #000000;
            }
            
            .price {
                font-size: 14px;
            }
            
            .up {
                font-size: 12px;
                color: #228B22;
            }
            
            .down {
                font-size: 12px;
                color: #D2042D;
            }
        </style>
    
        <div class="container mt-5">
            <div class="row">
                <div class="col-md-12">
                    <div class="d-flex justify-content-between align-items-center breaking-news bg-white">
                        <marquee class="news-scroll" behavior="scroll" direction="left" onmouseover="this.stop();" onmouseout="this.start();">""" + 
                            ' '.join(scroll_objects) + 
                        """</marquee>
                    </div>
                </div>
            </div>
        </div>
    
    </html>
    """.strip()
)

##### FAANG and ETF Prices and Candlestick

In [32]:
COLUMNS = 5
TICKERS = [
    'SPY', 'QQQ', 'XLF', 'IWM', 'BND',
    'FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG',
]

In [33]:
big_numbers = []
figures = []

for i, t in enumerate(TICKERS):
    close, delta, delta_pct, high, low, open_, p_close, _ = quote(FINNHUB_KEY, t)
    df = candles(FINNHUB_KEY, t)
    
    bn = dp.BigNumber(
        heading=t,
        value=f"${round(close, 2)}",
        change=f"{round(delta_pct, 2)}%",
        is_upward_change=True if delta_pct > 0 else False
    )
    big_numbers.append(bn)
    
    figure = candlestick(df, t)
    figures.append(figure)

In [34]:
ticker_groups = list(zip_longest(*(iter(big_numbers),) * COLUMNS, fillvalue=''))
ticker_groups = [
    dp.Group(columns=COLUMNS, *g) for g in ticker_groups
]

##### Scrape CNN Market

In [35]:
cnn = 'https://money.cnn.com/data/markets'
with requests.get(cnn) as page:
    soup = BeautifulSoup(page.content, 'html.parser')

In [36]:
modules = soup.find_all('div', class_='module')
key_stats = soup.find('ul', class_='module-body wsod key-stats')

In [37]:
pairs = list(map(format_modules, modules))
pairs = list(filter(None, pairs))
pairs = list(sum(pairs, ()))

In [38]:
headers = pairs[::2]
modules = pairs[1::2]

In [39]:
module_groups = list(map(group_modules, headers, modules))

##### Free Stock Search

In [40]:
%%time

# sp500_figures = free_stock_search(FINNHUB_KEY)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


#### 1.2.4 Portfolio Performance <a class="anchor" id="robinhood"></a>

Display Holdings

In [41]:
ticks, data, portfolio = load_portfolio(robinhood)

In [42]:
equity_tickers, etf_tickers, crypto_tickers = ticks
tickers = equity_tickers + etf_tickers + crypto_tickers

In [43]:
portfolio_toggles = list(
    map(
        lambda t, l: ticker_toggle(FINNHUB_KEY, t, l) if len(t) != 0 else None, 
        [equity_tickers, etf_tickers, crypto_tickers],
        ['Equity', 'ETF', 'Crypto']
    )
)
portfolio_toggles = list(filter(None, portfolio_toggles))

Overall Portfolio

In [44]:
crypto_prices = np.array([
    float(robinhood.crypto.get_crypto_quote(t)['mark_price']) for t in crypto_tickers
])
crypto_quantity = data[2].quantity.astype(float).to_numpy()
crypto_value = (crypto_prices * crypto_quantity)[0]

In [45]:
mkt_value, prev_mkt_value = portfolio['market_value'], portfolio['last_core_market_value']
mkt_value, prev_mkt_value = float(mkt_value), float(prev_mkt_value)
delta_pct = mkt_value / prev_mkt_value - 1

In [46]:
cash = float(portfolio['withdrawable_amount'])

In [47]:
portfolio_value = mkt_value + crypto_value + cash

In [48]:
portfolio_value = dp.BigNumber(
    heading="Overall Portfolio",
    value=f"${round(portfolio_value, 2)}",
    change=f"{round(delta_pct, 2)}%",
    is_upward_change=True if delta_pct > 0 else False
)
cash = dp.BigNumber(
    heading="Buying Power",
    value=f"${round(cash, 2)}"
)

Portfolio News

In [85]:
portfolio_news = list(map(
    lambda t: robinhood_news(robinhood, t), equity_tickers
))
portfolio_news = [
    n for news in portfolio_news for n in news 
]

Miscellaneous

In [50]:
# robinhood.markets.get_market_next_open_hours('XNYS')

In [51]:
TAGS = [
    'technology', 'banking', 'energy'
]

In [52]:
# s = robinhood.markets.get_all_stocks_from_market_tag('energy')
# s = pd.DataFrame(s) #.sort_values(by='last_trade_price', ascending=False)
# # print(len(s))

In [53]:
# movers = robinhood.markets.get_top_movers()
# movers = pd.DataFrame(movers)
# movers

In [54]:
# movers_u = robinhood.markets.get_top_movers_sp500(direction='up')
# movers_d = robinhood.markets.get_top_movers_sp500(direction='down')
# movers_u = pd.DataFrame(movers_u)
# movers_d = pd.DataFrame(movers_d)

Portfolio Candlesticks

In [55]:
portfolio_candles = list(map(
    partial(candles, FINNHUB_KEY), tickers
))
portfolio_figures = list(map(
    candlestick, portfolio_candles, tickers
))

#### 1.2.5 Data Exploration and Visualization <a class="anchor" id="data"></a>

##### Five Thirty Eight

In [56]:
NBA_ELO = 'https://projects.fivethirtyeight.com/nba-model/nba_elo.csv'
NFL_ELO = 'https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv'
MLB_ELO = 'https://projects.fivethirtyeight.com/mlb-api/mlb_elo.csv'
ELO_LINKS = [
    NBA_ELO, NFL_ELO, MLB_ELO
]

In [57]:
YEAR = 2021
NCAA = f'https://projects.fivethirtyeight.com/march-madness-api/{YEAR}/fivethirtyeight_ncaa_forecasts.csv'
POLLS = 'https://github.com/fivethirtyeight/data/tree/master/polls'

In [58]:
%%time

_538_figures = list(map(
    plot_elo, ELO_LINKS
))

CPU times: user 13.4 s, sys: 816 ms, total: 14.2 s
Wall time: 18.8 s


In [59]:
# TODO Clean and Style Graphs
    # Restyle Color
# TODO Header Blocks --> Intro ELO and By Sport
# Incorporate March Madness and ~Polls~

##### Spotify

In [60]:
TOP50 = '37i9dQZEVXbLRQDuF5jeBp'
MIX1 = '37i9dQZF1E35bNzojVbMHG'
MIX2 = '37i9dQZF1E38sJU7OsGSQi'
MIX3 = '37i9dQZF1E37xnKwvD1GJE'

PLAYLISTS = [
    TOP50, MIX1, MIX2, MIX3
]

In [61]:
HEADERS = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [62]:
playlists_html = list(map(get_spotify_embed, PLAYLISTS))

In [63]:
# TODO Correct Embedding...

##### Lichess

In [64]:
# TODO Top Player Matches

##### COVID

US KPIs

In [65]:
usa_link = f'https://api.covidactnow.org/v2/country/US.json?apiKey={COVID_KEY}'
with requests.get(usa_link) as r:
    usa = r.json()

In [66]:
stats = usa['actuals']

In [67]:
c, n_c, d, n_d = stats['cases'], stats['newCases'], stats['deaths'], stats['newDeaths']
vax_rat = usa['metrics']['vaccinationsCompletedRatio']

In [68]:
covid_kpi = dp.Group(
    dp.BigNumber(
        heading='Total Cases',
        value="{:,}".format(c),
        change=f"{round(n_c / c * 100, 2)}%",
        is_upward_change=True if n_c / c - 1 > 0 else False
    ),
    dp.BigNumber(
        heading='Total Deaths',
        value="{:,}".format(d),
        change=f"{round(n_d / d * 100, 2)}%",
        is_upward_change=True if n_d / d - 1 > 0 else False
    ),
    dp.BigNumber(
        heading='Vaccine Completion',
        value=f"{round(vax_rat * 100, 2)}%"
    ),
    columns=3
)

County Choropleth

In [69]:
geo_link = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
with requests.get(geo_link) as r:
    geo_json = r.json()

In [70]:
counties = f'https://api.covidactnow.org/v2/counties.json?apiKey={COVID_KEY}'

In [71]:
with requests.get(counties) as r:
    counties = pd.DataFrame(r.json())

In [72]:
counties['cases'] = counties.actuals.str['cases']
counties['case_ratio'] = counties.cases / counties.population
counties['deaths'] = counties.actuals.str['deaths']
counties['death_ratio'] = counties.deaths / counties.population
counties['vaccine_ratio'] = counties.metrics.str['vaccinationsCompletedRatio']

In [73]:
%%time

covid_fig = county_choropleth(counties, geo_json)

CPU times: user 2.1 s, sys: 347 ms, total: 2.45 s
Wall time: 3.37 s


Individual State Plots

In [74]:
states = f'https://api.covidactnow.org/v2/states.timeseries.json?apiKey={COVID_KEY}'

In [75]:
with requests.get(states) as r:
    states = pd.DataFrame(r.json())

In [76]:
%%time

covid_figures = list(map(
    plot_state, states.state, states.actualsTimeseries
))

CPU times: user 18.9 s, sys: 294 ms, total: 19.2 s
Wall time: 19.3 s


##### Data Exploration Compilation

In [77]:
_538_group = dp.Toggle(
    dp.Select(
        *_538_figures
    ),
    label = 'Five Thirty Eight'
)
lichess_group = dp.Toggle(
    dp.Text("Lichess", label='A'),
    label = 'Lichess'
)
spotify_group = dp.Toggle(
    dp.Group(
        *playlists_html,
        columns=4
    ),
    label = 'Spotify'
)
covid_group = dp.Toggle(
    covid_kpi,
    dp.Divider(),
    dp.Plot(
        covid_fig
    ),
    dp.Divider(),
    dp.Select(
        *covid_figures
    ),
    label = 'COVID'
)

In [78]:
data_groups = [
    covid_group,
    _538_group
#     spotify_group,
#     lichess_group,
]

#### 1.2.6 Compiled Selector <a class="anchor" id="compiled"></a>

In [86]:
sports = dp.Group(
    dp.Select(
        *espn_groups,
        type=dp.SelectType.DROPDOWN
    ),
    label='Sports: Results and Analysis',
)
news = dp.Group(
    dp.Select(*article_sections),
    label='Top World News'
)
market = dp.Group(
    scroll,
    dp.Divider(),
    *ticker_groups,
    dp.Divider(),
    dp.Select(
        *figures
    ),
    dp.Divider(),
    *module_groups[:2],
    dp.Group(
        *module_groups[2:],
        columns=4
    ),
#     dp.Select(
#         *sp500_figures
#     ),
    label='Stock Market'
)
portfolio = dp.Group(
    dp.Group(
        portfolio_value,
        cash,
        columns=2
    ),
    dp.Divider(),
    *portfolio_toggles,
    dp.Divider(),
    dp.Select(
        *portfolio_figures
    ),
    dp.Divider(),
    dp.Group(
        *portfolio_news,
        columns=2
    ),
    label='Robinhood Portfolio'
)
data = dp.Group(
    *data_groups,
    label='Data Exploration'
)

In [87]:
main_select = dp.Select(
    blocks=[
        sports, market # news, # market, # portfolio #, data

    ],
    type=dp.SelectType.TABS,
    label='selector'
)

### 1.3 Credits <a class="anchor" id="credits"></a>

[Table of Contents](#toc)

In [88]:
credits = dp.Text(
    "Report built by Iain Muir.",
    name='credits'
)

## 2. Datapane Report <a class="anchor" id="report"></a>

[Table of Contents](#toc)

In [113]:
BLOCKS = [
    header_logo,
    header_text,
    header_description,
    dp.Divider(),
    sports,
#     main_select,
    dp.Divider(),
    credits
]

In [114]:
report = dp.Report(
    blocks=BLOCKS
)

In [116]:
try:
    report.upload(
        name='Information Aggregator', 
        open=False
    )
except requests.exceptions.HTTPError:
    print(
        ErrorHandler("Report Upload Error; FATAL.", *get_error_info())
    )
    time.sleep(5)
    
    report.upload(
        name='Information Aggregator', 
        open=False
    )

In [ ]:
report.save(
    path=f'{ROOT}/Output/Information-Aggregator.html'
)

In [105]:
webbrowser.open(
    report.web_url
)

True

In [ ]:
Logging.write_success_to_log()